# Integración de Datos Datawarehouse

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hermetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd '/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module'

/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module


## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein
import re

# Ruta de archivos

In [3]:
base_path = os.getcwd()
path = os.path.join(base_path, "CLEANED_DATA")
path_datawarehouse = os.path.join(base_path, "DATAWAREHOUSE")
path_datamart_oc = 'datamart_oc'
path_datamart_oec = 'datamart_oec'
path_datamart_ci = 'datamart_ci'

 ## Tabla Ubicacion

In [4]:
df_ubicacion = pd.read_csv(os.path.join(path_datawarehouse, 'ubicacion.csv'))

# Funciones de procesamiento de datos

In [6]:
year_list = [str(x) for x in range(2022,2009,-1)]

def get_year(x):
    for year in year_list:
        if re.search('-'+year+'\s|'+year+'-|-'+year+'$',x):
            return year
    return 'sin_anio'

In [7]:
def get_year_by_oc(x,y):
    if x=='sin_anio':
        if y != 'Sin Fecha Resolución' and not pd.isna(y):
            return y[-4:]
    else:
        return x

## Dimensión Razon Social OC

In [64]:
df_oc = pd.read_csv(os.path.join(path,'oc.csv'))
df_oc_ug = pd.read_csv(os.path.join(path,'oc_dl_provincia_canton.csv'))
df_oc_ug['estado']='desconocido'
df_razon_social_oc = pd.concat([df_oc[['razon_social','estado','canton']],df_oc_ug[['razon_social','estado','canton']]],axis=0)
df_razon_social_oc = df_razon_social_oc.drop_duplicates(subset=['razon_social','canton'],keep='first')
df_razon_social_oc=df_razon_social_oc.rename(columns={'razon_social':'nombre'})
df_razon_social_oc = pd.merge(df_razon_social_oc,df_ubicacion,on='canton',how="left")
df_razon_social_oc.pop('id_ubicacion')
df_razon_social_oc = df_razon_social_oc.fillna({'provincia':'sin provincia'})
df_razon_social_oc = df_razon_social_oc.rename_axis('id_oc').reset_index().astype('object')
df_razon_social_oc = df_razon_social_oc.apply(lambda x: x.astype(str).str.lower())

In [54]:
#df_razon_social_oc.head()
#df_razon_social_oc.info()

In [65]:
df_razon_social_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_razon_social_oc.csv'),index=False)

## Dimensión Curso_Perfil

In [56]:
df_oc_cl = pd.read_csv(os.path.join(path,'oc_dl_familia_sector_perfil.csv'))
df_oc_cl_2 = pd.read_csv(os.path.join(path,'oc_cl.csv'))
df_oc_cl_2.pop('RUC_o_Codigo')
df_oc_cl_2 = df_oc_cl_2.rename(columns={'Razon_Social':'Nombre'})
df_oc_cl = pd.concat([df_oc_cl,df_oc_cl_2],axis=0)

In [57]:
df_oc_cl = df_oc_cl.drop_duplicates()
df_oc_cl['modalidad'] = 'desconocida'
df_oc_cl['carga_horaria'] = np.NaN
df_oc_cl['tipo'] = 'perfil'
df_oc_cc = pd.read_csv(os.path.join(path,'oc_cc.csv'))
df_oc_cc.pop('documento')
df_oc_cc['tipo'] = 'curso'

In [58]:
data_curso_perfil = np.concatenate((df_oc_cc.values,df_oc_cl.values), axis=0)
df_curso_perfil = pd.DataFrame(data=data_curso_perfil,columns=['razon_social','area_familia','especialidad_sector','curso_perfil','modalidad','carga_horaria','tipo'])
df_curso_perfil.insert(0,'tipo',df_curso_perfil.pop('tipo'))
df_curso_perfil = df_curso_perfil.rename_axis('id_curso_perfil').reset_index().astype('object')
df_curso_perfil = df_curso_perfil.apply(lambda x: x.astype(str).str.lower())
df_curso_perfil['carga_horaria'] = df_curso_perfil['carga_horaria'].astype('float64')

In [59]:
#df_curso_perfil.head()
#df_curso_perfil.info()

## Hechos OC

In [70]:
#AGREGAR AÑO
# Cargar los archivos necesarios de la ubicación de CLEANED DATA
df_pcoc = pd.read_csv(os.path.join(path,'pcoc.csv'))
df_pcoc = df_pcoc.apply(lambda x: x.astype(str).str.lower())

# Se extraera el año de la columna Numero_certificado.
# Para esto se buscará porciones de texto que contengan los años del 2000 - 2022 (año actual)

df_pcoc['anio'] = df_pcoc['Numero_Certificado'].apply(get_year)

df_oc = pd.read_csv(os.path.join(path,'oc.csv'))

df_pcoc = pd.merge(df_pcoc,df_oc[['razon_social','fecha_resolucion']],left_on=['Razon_Social_OC'],right_on=['razon_social'],how="left") 


df_pcoc['anio'] = df_pcoc.apply(lambda x: get_year_by_oc(x['anio'], x['fecha_resolucion']), axis=1)

df_pcoc[
    (df_pcoc['anio']=='sin_anio') & 
    (df_pcoc['fecha_resolucion']!='Sin Fecha Resolución') &
    (df_pcoc['fecha_resolucion'].notnull())
    ]['anio'] = df_pcoc[
    (df_pcoc['anio']=='sin_anio') & 
    (df_pcoc['fecha_resolucion']!='Sin Fecha Resolución') &
    (df_pcoc['fecha_resolucion'].notnull())
    ]['fecha_resolucion'].str.slice(start=-4)

df_pcoc = df_pcoc.drop(['razon_social','fecha_resolucion'],axis=1)


## Dimension Fecha OC

Se debe desarrollar en base a el año el cual se haya obtenido el certificado del curso o perfil OC.

En caso de no existir un año registrado en el certificado en la tabla **PCOC** se deberá tomar como referencia el año de resolución de la Razon social de la tabla **OC**

In [67]:
df_fecha_oc = pd.DataFrame(data=df_pcoc['anio'].unique(),columns=['anio'])
df_fecha_oc = df_fecha_oc.rename_axis('id_fecha').reset_index().astype('object')
df_fecha_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_fecha.csv'),index=False)

## Guardar Datamart OC

In [71]:
## UNIR DATOS
df_fact_oc = df_pcoc.groupby(['Razon_Social_OC','Nombre_Curso_Perfil','anio']).count().reset_index().iloc[:,:4]
df_fact_oc = df_fact_oc.rename(columns={'Numero_Documento':'num_cap_cer'})
df_fact_oc = pd.merge(df_fact_oc,df_razon_social_oc,left_on=['Razon_Social_OC'],right_on=['nombre'],how="left")
df_fact_oc = pd.merge(df_fact_oc,df_curso_perfil,left_on=['Razon_Social_OC','Nombre_Curso_Perfil'],right_on=['razon_social','curso_perfil'],how="left")
df_fact_oc = pd.merge(df_fact_oc,df_fecha_oc,left_on=['anio'],right_on=['anio'],how="left")


In [76]:
df_fact_oc = df_fact_oc[['id_oc','id_curso_perfil','id_fecha','num_cap_cer','carga_horaria']]
df_fact_oc['total_horas'] = df_fact_oc['num_cap_cer']*df_fact_oc['carga_horaria']
df_fact_oc.drop('carga_horaria',axis=1,inplace=True)

,id_oc,id_curso_perfil,id_fecha,num_cap_cer,total_horas
0,NaN,NaN,2,17,NaN
1,NaN,NaN,2,46,NaN
2,369,NaN,4,2,NaN
3,1128,NaN,4,2,NaN
4,369,NaN,4,15,NaN
...,...,...,...,...,...
9381,1207,NaN,4,8,NaN
9382,459,NaN,4,7,NaN
9383,1207,NaN,4,7,NaN
9384,459,NaN,4,8,NaN


In [77]:
#df_fact_oc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9386 entries, 0 to 9385
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_oc            9160 non-null   object 
 1   id_curso_perfil  3192 non-null   object 
 2   id_fecha         8829 non-null   object 
 3   num_cap_cer      9386 non-null   int64  
 4   total_horas      3106 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 440.0+ KB


In [78]:
df_curso_perfil.drop('razon_social',axis=1,inplace=True)
#df_curso_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_curso_perfil.csv'),index=False)

In [47]:
#df_fact_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'fact_oc.csv'),index=False)

# Dataframe OEC

### Dimensión Razon Social OEC

In [96]:
df_oec_2 = pd.read_csv(os.path.join(path,'oec_dl_provincia_canton.csv'))
df_oec_2['estado'] = 'desconocido'
df_oec_2 = df_oec_2[['razon_social','estado','canton']]
df_oec = pd.read_csv(os.path.join(path,'oec.csv'))
df_razon_social_oec = df_oec[['razon_social','estado','canton']]
df_razon_social_oec = pd.concat([df_razon_social_oec,df_oec_2])
df_razon_social_oec = pd.merge(df_razon_social_oec,df_ubicacion,on='canton',how="left")
df_razon_social_oec.pop('id_ubicacion')
df_razon_social_oec = df_razon_social_oec.fillna({'provincia':'sin provincia'})
df_razon_social_oec = df_razon_social_oec.drop_duplicates(subset=['razon_social'], keep='first')
df_razon_social_oec = df_razon_social_oec.rename_axis('id_oec').reset_index().astype('object')
df_razon_social_oec = df_razon_social_oec.apply(lambda x: x.astype(str).str.lower())

In [97]:
#df_razon_social_oec.head()
#df_razon_social_oec.info()

In [98]:
df_razon_social_oec.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_razon_social_oec.csv'),index=False)

### Dimension Perfil

In [99]:
df_perfil = pd.read_csv(os.path.join(path,'oec_dl_familia_sector_perfil.csv'))
df_perfil = df_perfil.rename(columns={'Nombre':'razon_social','Familia':'familia','Sector':'sector','Perfil':'perfil'})
df_perfil = df_perfil.rename_axis('id_perfil').reset_index().astype('object')
df_perfil['modalidad'] = 'desconocida'
df_perfil['carga_horaria'] = np.NAN
df_perfil = df_perfil.apply(lambda x: x.astype(str).str.lower())
df_perfil['carga_horaria'] = df_perfil['carga_horaria'].astype('float64')

In [100]:
#df_perfil.head()
#df_perfil.info()

In [101]:
df_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_perfil.csv'),index=False)

## Hechos OEC

In [102]:
df_pcoec = pd.read_csv(os.path.join(path,'pcoec.csv'))
df_pcoec = df_pcoec.apply(lambda x: x.astype(str).str.lower())
df_demanda_oec = df_pcoec.groupby(['OEC','Perfil','Fecha_Certificacion']).count().reset_index().iloc[:,:4]
df_demanda_oec = df_demanda_oec.rename(columns={'Nombres':'num_cer'})
df_demanda_oec = pd.merge(df_demanda_oec,df_razon_social_oec,left_on=['OEC'],right_on=['razon_social'],how="left")

## Dimension Fecha

In [103]:
df_fecha_oec = df_demanda_oec['Fecha_Certificacion'].drop_duplicates()
df_fecha_oec = df_fecha_oec.rename_axis('id_fecha').reset_index().astype('object')
data_fecha = df_fecha_oec['Fecha_Certificacion'].str.rsplit("-", expand=True)
df_fecha_oec = pd.concat([df_fecha_oec,data_fecha],axis=1)
df_fecha_oec = df_fecha_oec.rename(columns={0:'dia',1:'mes',2:'año'})

In [104]:
#df_fecha_oec.head()
#df_fecha_oec.info()

In [107]:
df_demanda_oec = df_pcoec.groupby(['OEC','Perfil','Fecha_Certificacion']).count().reset_index().iloc[:,:4]
df_demanda_oec = df_demanda_oec.rename(columns={'Nombres':'num_cer'})
df_demanda_oec = pd.merge(df_demanda_oec,df_razon_social_oec,left_on=['OEC'],right_on=['razon_social'],how="left")
df_demanda_oec = pd.merge(df_demanda_oec,df_perfil,left_on=['OEC','Perfil'],right_on=['razon_social','perfil'],how="left")
df_demanda_oec = pd.merge(df_demanda_oec,df_fecha_oec,left_on=['Fecha_Certificacion'],right_on=['Fecha_Certificacion'],how="left")
#df_demanda_oec = df_demanda_oec[['id_oec','id_perfil','id_fecha','num_cer']]

In [109]:
#df_demanda_oec.head()
#df_demanda_oec.info()

In [110]:
df_fecha_oec = df_fecha_oec.drop('Fecha_Certificacion',axis=1)
df_fecha_oec.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_fecha.csv'),index=False)

In [111]:
df_perfil = df_perfil.drop('razon_social',axis=1)
df_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'dim_perfil.csv'),index=False)

In [112]:
df_demanda_oec.to_csv(os.path.join(path_datawarehouse,path_datamart_oec,'fact_oec.csv'),index=False)

# Dataframe CI

## Dimensión Capacitador independiente

In [8]:
df_capacitador_ci = pd.read_csv(os.path.join(path,'ci.csv'))
df_capacitador_ci = df_capacitador_ci.rename(columns={'Apellidos_Nombres':'razon_social','Codigo_Resolucion':'codigo_resolucion'})
df_capacitador_ci = df_capacitador_ci[['razon_social','codigo_resolucion']]
df_capacitador_ci_2 = pd.read_csv(os.path.join(path,'ci_dl_provincia_canton.csv'))
df_capacitador_ci = pd.merge(df_capacitador_ci,df_capacitador_ci_2,left_on=['razon_social'],right_on=['razon_social'],how="left")
df_capacitador_ci_2.insert(1,'codigo_resolucion','DESCONOCIDO')
df_capacitador_ci = pd.concat([df_capacitador_ci,df_capacitador_ci_2],axis=0)
df_capacitador_ci = df_capacitador_ci.drop_duplicates(subset=['razon_social','provincia','canton'],keep='first')
df_capacitador_ci = df_capacitador_ci.fillna({'provincia':'sin provincia','canton':'sin canton'})
df_capacitador_ci['anio'] = df_capacitador_ci['codigo_resolucion'].apply(get_year)
df_capacitador_ci  = df_capacitador_ci.rename_axis('id_ci').reset_index().astype('object')
df_capacitador_ci = df_capacitador_ci.apply(lambda x: x.astype(str).str.lower())

In [11]:
#df_capacitador_ci.head()
#df_capacitador_ci.info()

### Dimesión Fecha CI

In [12]:
df_fecha_ci = pd.DataFrame(data=df_capacitador_ci['anio'].unique(),columns=['anio'])
df_fecha_ci = df_fecha_ci.rename_axis('id_fecha').reset_index().astype('object')

## Dimensión Curso

In [13]:
df_ci_lca = pd.read_csv(os.path.join(path,'ci_lca.csv'))
df_ci_lca = df_ci_lca.rename(columns={'Nombre_Curso':'curso','Area':'area','Especialidad':'especialidad','Carga Horaria':'carga_horaria'})
df_ci_lca =  df_ci_lca.rename_axis('id_curso').reset_index().astype('object')
df_ci_lca = df_ci_lca.apply(lambda x: x.astype(str).str.lower())
df_ci_lca['carga_horaria'] =  df_ci_lca['carga_horaria'].astype('float')

In [15]:
#df_ci_lca.head()
#df_ci_lca.info()

## Hechos CI

In [16]:
df_pcci = pd.read_csv(os.path.join(path,'pcci.csv'))
df_pcci = df_pcci.apply(lambda x: x.astype(str).str.lower())
df_pcci = pd.merge(df_pcci,df_capacitador_ci,left_on=['Instructor_Capacitador'],right_on=['razon_social'],how="left")
df_pcci = df_pcci.groupby(['Instructor_Capacitador','Nombre_Curso','anio']).count().reset_index()[['Instructor_Capacitador','Nombre_Curso','anio','Numero_Documento']]
df_pcci = df_pcci.rename(columns={'Numero_Documento':'num_cap'})
df_pcci = pd.merge(df_pcci,df_ci_lca,left_on=['Instructor_Capacitador','Nombre_Curso'],right_on=['Apellidos_Nombres','curso'],how="left")
df_pcci = pd.merge(df_pcci,df_fecha_ci,left_on=['anio'],right_on=['anio'],how="left")
df_pcci = pd.merge(df_pcci,df_capacitador_ci,left_on=['Instructor_Capacitador'],right_on=['razon_social'],how="left")
fact_ci = df_pcci[['id_ci','id_curso','id_fecha','num_cap','carga_horaria']]
fact_ci['total_horas'] = fact_ci['num_cap']*fact_ci['carga_horaria']

C:\Users\sjime\AppData\Local\Temp\ipykernel_21228\3007085897.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_ci['total_horas'] = fact_ci['num_cap']*fact_ci['carga_horaria']


In [19]:
fact_ci.drop('carga_horaria',axis=1,inplace=True)

C:\Users\sjime\AppData\Local\Temp\ipykernel_21228\3183824890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_ci.drop('carga_horaria',axis=1,inplace=True)


In [21]:
#fact_ci.head()
#fact_ci.info()

In [22]:
df_ci_lca = df_ci_lca.drop(['Numero_Documento', 'Apellidos_Nombres'],axis=1)
df_ci_lca.insert(3,'curso',df_ci_lca.pop('curso'))
df_ci_lca.insert(4,'modalidad','desconocida')
df_ci_lca = df_ci_lca.apply(lambda x: x.astype(str).str.lower())
df_ci_lca['carga_horaria'] = df_ci_lca['carga_horaria'].astype('float64')


In [25]:
#df_ci_lca.head()
#df_ci_lca.info()

### Guardar Datamart

In [26]:
df_ci_lca.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'dim_curso.csv'),index=False)
df_capacitador_ci.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'dim_capacitador_independiente.csv'),index=False)
df_fecha_ci.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'dim_fecha.csv'),index=False)
fact_ci.to_csv(os.path.join(path_datawarehouse,path_datamart_ci,'fact_ci.csv'),index=False)

# Costos

In [53]:
lev = Levenshtein()

In [56]:
stopwords = open(os.path.join(path,'spanish.txt'),"r", encoding='utf-8')
stopwords = stopwords.readlines()
stopwords = [x.replace('\n','') for x in stopwords]

In [57]:
def normalizar(text):
    puntuacion = '!"#$%&;\'()*+,-./?@[\]^_`{|}~()'
    text = text.lower() 
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.strip()
    text = text.split(' ')
    re_punc = re.compile('[%s]' % re.escape(puntuacion))
    stripped = [re_punc.sub(' ', w) for w in text]
    text = ' '.join(stripped)
    text = text.split(' ')
    new_text = []
    for i in range(len(text)):
        char = text[i]
        if char not in stopwords:
            new_text.append(text[i])
    new_text = ' '.join(new_text)
            
    return new_text

In [59]:
#Filtrar las areas de Tecnologia y TICs
df_dim_curso_perfil = pd.read_csv(os.path.join(path_datawarehouse,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[(df_dim_curso_perfil['area_familia']=='información y comunicación')|(df_dim_curso_perfil['area_familia'] == 'tecnologías de la información y comunicación')]
data_1= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data_1 = data_1[['curso_perfil','modalidad']]

df_dim_perfil = pd.read_csv(os.path.join(path_datawarehouse,'datamart_oec','dim_perfil.csv'))
df_dim_perfil = df_dim_perfil[df_dim_perfil['familia']=='tecnología: hardware y software (incluye tic)']
data_2 = df_dim_perfil.groupby('perfil').count().reset_index()
data_2['modalidad'] = 'desconocida'
data_2 = data_2[['perfil','modalidad']]

df_dim_curso = pd.read_csv(os.path.join(path_datawarehouse,'datamart_ci','dim_curso.csv'))
df_dim_curso = df_dim_curso[df_dim_curso['area']=='tecnologías de la información y comunicación']
data_3 = df_dim_curso.groupby('curso').count().reset_index()
data_3['modalidad'] = 'desconocida'
data_3 = data_3[['curso','modalidad']]

arr_dims_curso_perfil = [df_dim_curso_perfil,df_dim_perfil,df_dim_curso]

In [60]:
data = np.concatenate((data_1.values,data_2.values,data_3.values),axis=0)
data = pd.DataFrame(data=data,columns=['curso_perfil','modalidad'])
data = data.drop_duplicates()
data['aux_2_curso_perfil'] = data['curso_perfil'].str.cat(data['modalidad'],sep=' | ')
data['aux_curso_perfil'] = data['curso_perfil'].apply(lambda x: normalizar(x))

In [62]:
df_coec = pd.read_csv(os.path.join(path,'costos_cursos_educacion_continua.csv'))
df_ccce = pd.read_csv(os.path.join(path,'costos_de_cursos_cec_epn.csv'))
df_coec = pd.concat([df_coec,df_ccce])
df_coec['aux_2_curso'] = df_coec['curso'].str.cat(df_coec['modalidad'],sep=' | ')
df_coec['aux_curso'] = df_coec['curso'].apply(lambda x: normalizar(x))

In [63]:
arr = []

for i in range(len(df_coec['aux_curso'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['aux_curso_perfil'])):
        curso_fuente = df_coec['aux_curso'].iloc[i]
        curso_comparar = data['aux_curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar,(1,1.5,1.25))
        aux_similary=0
        arr_curso = curso_comparar.split(' ')
        if curso_comparar in curso_fuente :
            similary = 1.0
        else:
            for word in arr_curso:
                if word in curso_fuente:
                    aux_similary+=1/len(arr_curso)
            if data['modalidad'].iloc[j] == df_coec['modalidad'].iloc[i]:
              aux_similary += aux_similary/2
        if aux_similary > similary and aux_similary > 2/len(arr_curso):
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [64]:
res = pd.DataFrame(data=arr,index=df_coec['aux_2_curso'],columns=data['aux_2_curso_perfil'])

In [ ]:
arr_dims_cols = ['curso_perfil','perfil','curso']
arr_datamarts = ['datamart_oc','datamart_oec','datamart_ci']
arr_files = ['dim_curso_perfil.csv','dim_perfil.csv','dim_curso.csv']
arr_df_final_dims = []
for i in range(len(arr_dims_curso_perfil)):
  df_dim = arr_dims_curso_perfil[i]
  df_dim['curso_modalidad'] = df_dim[arr_dims_cols[i]].str.cat(df_dim['modalidad'],sep=' | ')
  arr_curso_fuente = []
  for j in range(arr_dims_curso_perfil[i].shape[0]):
      query = df_dim['curso_modalidad'].iloc[j]
      val = res[res[query]==res[query].max()][query].index.values[0]
      arr_curso_fuente.append(df_coec[df_coec['aux_2_curso']==val][['curso','precio_por_hora']].values[0])
  df_curso_fuente_precio = pd.DataFrame(data=arr_curso_fuente,columns=['curso_fuente','precio_por_hora'])
  df_res_corr = pd.DataFrame(data=[df_dim[arr_dims_cols[i]].values,df_dim['modalidad'].values,df_curso_fuente_precio['curso_fuente'].values,df_curso_fuente_precio['precio_por_hora'].values])
  df_res_corr = df_res_corr.transpose()
  df_res_corr = df_res_corr.rename(columns={0:arr_dims_cols[i],1:'modalidad',2:'curso_fuente',3:'precio_por_hora'})
  df_dim = pd.read_csv(os.path.join(path_datawarehouse,arr_datamarts[i],arr_files[i]))
  df_dim= pd.merge(df_dim,df_res_corr,left_on=[arr_dims_cols[i],'modalidad'],right_on=[arr_dims_cols[i],'modalidad'],how='left')
  df_dim['costo'] = df_dim['carga_horaria']*df_dim['precio_por_hora']
  df_dim = df_dim.drop(['curso_fuente','precio_por_hora'], axis=1)
  df_dim.to_csv(os.path.join(path_datawarehouse,arr_datamarts[i],arr_files[i]),index=False)
  arr_df_final_dims.append(df_dim)
